# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane.

**Simulations**:
- **CTRL**: fixed surface concentration of hydrogen and methane
- **10H2**: as CTRL, but with surface H2 increased by 10%. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.


AGWP (as defined as the integral over the radiative forcing of a pulse for a given time horizon, e.g. 100 years) is equal to the steady-state radiative forcing (W m-2) divided by the emissions (Tg-H2 yr-1). Based on the three set of simulations, we calculate the radiative forcing in the perturbed relative to the control. As the models run with fixed surface concentration of methane, we need to have a separate perturbed methane run. 

We calculate the radiative forcing per Tg-CH4 (including the feedbacks) for the methane perturbed run, and map the changes in the methane loss in the hydrogen perturbation with the results from the methane perturbed run. 

In [1]:
%%html
<style>
table {float:left}
</style>


| Models   | Simulations length | Average period |
|  :---    | :----              | :---           |
| OsloCTM3 | 20 years           |   year 20      |
| WACCM6   | 18 years           | last 5 years   |
| INCA     | 10 years           | last 3 years   |
| GFDL*    | 50 years           | last 30 years  | 





*GFDL simulations from Paulot et al. (2021) with H2 emissions (200 Tg/yr).*
Monthly mean data.

#### OUTLINE:
**PART I: Read model results**

1. Hydrogen budget
    1. H2 burden
    2. H2 loss
    3. H2 production
    4. Estimated H2 emissions
    5. H2 surface concentrations
    6. H2 lifetime
    7. H2 flux
2. Methane budget
    1. CH4 burden
    2. CH4 atmospheric loss
    3. CH4 surface concentrations
    4. CH4 lifetime
    5. CH4 flux
3. Ozone
    1. Change in troposheric ozone
    2. Change in stratosheric ozone
    3.  
    4. 
    5. Ozone radiative forcing
4. Stratospheric water wapor

**PART II: GWP calculations**


**PART III: Main results and tabels**

**Appendix with additional results**

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [3]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [4]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: Table 7.SM.6 in IPCC AR6: 0.0895 pW m-2 yr kg-1 (p=10^-12) 
#agwp100_CO2 = 0.09170  AR5 value.
agwp100_CO2 = 0.0895
##agwp100_CO2 = 0.0243 #AGWP_20 make separate notebook
##agwp100_CO2 = 0.314 #AGWP_500 make separate notebook


#CH4 tau_strat[yr] Source: REF
tau_strat = 120.0

#CH4 tau_soil [yr] Source: REF
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44300

**Dry deposition adjustments** As there is large uncertanty in dry deposition, we set the dry deposition values in the control run to a high and a low value based on literature (90 and 50 Tg/yr). This will result in a range of GWP values. Adjust the perturbations with the same relative factors as in the control. 

In [5]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 90.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

# Part I: Read model results

In this part, model results are read from the input files.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [6]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,196,195,195,192,209,191,188
10H2,215,213,213,659,240,210,261
10CH4,197,195,195,nan,nan,192,188
deltaH2,18.7,18.7,18.7,467,31.1,19.1,73.0


### 1.2. H2 loss
Hydrogen loss happens through two main processes. The largest loss is through dry deposition at the ground. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 dry deposition [Tg/yr]

The models diagnose drydeposition based on their own schemes, but this is not used directly in the concentration driven runs, as the surface concentrations are fixed.

In [7]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,59.5,73.0,52.0,54.6,58.4,88.7,56.6
10H2,65.5,80.3,57.1,204,67.8,97.5,79.2
10CH4,59.5,73.0,52.0,nan,0.00,88.7,56.6


As drydeposition is uncertain, we can replace the models diagnosed drydeposition by a given value. In the concentration driven run, the drydeposition scheme do not impact the atmospheric composition. For emission driven runs, we also do adjust the drydeposition. Remark that we do not use the emission numbers from the emission driven runs directly, the fluxes is estimated based on burden and lifetimes.

In [8]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

#### H2 atmospheric loss [Tg/yr]

In [9]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,28.4,29.0,22.5,22.3,30.1,26.8,21.7
10H2,31.0,31.7,24.5,70.2,34.4,29.2,29.7
10CH4,27.6,28.2,22.0,nan,0.00,26.0,21.2


#### H2 total loss [Tg/yr]:

In [10]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,87.9,102,74.4,77.0,88.4,115,78.3
10H2,96.4,112,81.7,274,102,127,109
10CH4,87.2,101,73.9,nan,0.00,115,77.8


### 1.3. H2 production

#### H2 atm. prod [Tg/yr]

In [11]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,56.3,33.9,47.1,45.1,56.2,49.4,45.0
10H2,56.2,33.8,47.0,43.5,56.1,49.3,44.8
10CH4,59.2,36.4,49.4,nan,0.00,51.9,47.1


### 1.4. Estimated H2 emissions (Total loss = Total prod + emission)

In surface concentration driven runs, there are two unknowns; emissions and dry deposition. 
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. We adjust the dry deposition to take into account uncertainties in emissions and dry deposition on the GWP values.

In [12]:
df_h2_estemis = df_h2_atmloss.loc['CTRL'] + df_h2_drydep.loc['CTRL'] - df_h2_atmprod.loc['CTRL']
df_h2_estemis

OSLOCTM3       31.6
WACCM6-2deg    68.1
INCA           27.3
GFDL-emi       31.9
OSLOCTM3-emi   32.2
UKCA           66.0
GFDL_nudge     33.3
Name: CTRL, dtype: float64

### 1.5. H2 surface concentration [ppb]

In [13]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,532,532,532,542,567,537,533
10H2,585,585,585,"1,912",656,590,746
deltaH2,53.2,53.2,53.2,"1,370",89.1,53.5,213


### 1.6. H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

In [14]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,2.23,1.91,2.61,2.50,2.36,1.66,2.40
10H2,2.23,1.91,2.61,2.41,2.35,1.66,2.40
10CH4,2.26,1.93,2.64,nan,nan,1.68,2.42


#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided only by the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [15]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,6.91,6.70,8.66,8.62,6.95,7.15,8.66
10H2,6.94,6.73,8.69,9.39,6.98,7.20,8.80
10CH4,7.12,6.93,8.88,nan,nan,7.40,8.90


#### H2 soil sink lifetime [yr]

In [16]:
df_h2_soil_sink_lifetime = df_h2_burden.drop('deltaH2')/df_h2_drydep
df_h2_soil_sink_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,3.30,2.67,3.74,3.52,3.58,2.16,3.32
10H2,3.28,2.66,3.73,3.23,3.54,2.16,3.30
10CH4,3.31,2.67,3.75,nan,nan,2.17,3.33


### 1.7. H2 flux (burden dived by lifetime) [Tg/yr]

The hydrogen flux is calculated as the burden divided by the hydrogen lifetime. The hydrogen lifetime is calculated by the burden divided by the total lifetime (drydep + atm.loss). (This is equal to the total loss, as it will be in steady state)

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks (add why). 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [17]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,87.9,102,74.4,77.0,88.4,115,78.3
10H2,96.4,112,81.7,274,102,127,109
10CH4,87.2,101,73.9,nan,nan,115,77.8
deltaH2,8.54,9.93,7.23,197,13.8,11.3,30.5
deltaCH4,-0.75,-0.86,-0.50,nan,nan,-0.75,-0.52


Description of the negative deltaCH4. In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

## 2. Methane results

### 2.1 CH4 burden [Tg]

In [18]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,"4,995","5,003","5,003","4,955","4,995","4,921","4,876"
10H2,"4,995","5,003","5,003","4,954","4,995","4,921","4,876"
10CH4,"5,496","5,505","5,504",nan,nan,"5,414","5,364"
deltaCH4,501,502,501,nan,nan,494,488


### 2.2 CH4 atmospheric loss [Tg/yr]

In [19]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,683,727,593,540,681,632,528
10H2,680,724,590,499,677,629,520
10CH4,728,772,635,nan,nan,672,564


### 2.3 CH4 surface concentration [ppb]

In [20]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,"1,813","1,808","1,813","1,832","1,813","1,806","1,816"
10CH4,"1,994","1,989","1,994",nan,nan,"1,986","1,998"
deltaCH4,181,181,181,nan,nan,181,182


### 2.4 CH4 lifetime [yr]

In [21]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,7.31,6.88,8.44,9.17,7.33,7.79,9.24
10H2,7.34,6.91,8.47,9.93,7.38,7.82,9.38
10CH4,7.55,7.13,8.67,nan,nan,8.05,9.51


#### Total CH4 lifetime [yr]

In [22]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,6.61,6.25,7.52,8.09,6.62,6.99,8.14
10H2,6.63,6.28,7.54,8.68,6.66,7.02,8.25
10CH4,6.80,6.46,7.70,nan,nan,7.21,8.35


### 2.5 CH4 flux (burden dived by total lifetime) [Tg/yr]

In [23]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,756,800,666,613,754,704,599
10H2,753,797,663,571,750,701,591
10CH4,808,852,715,nan,nan,751,642
deltaH2,-2.67,-3.28,-2.12,-41.7,-4.39,-2.76,-7.96
deltaCH4,52.4,52.4,49.4,nan,nan,47.6,43.4


## 3. Ozone burden and RF

### 3.1 Change in tropospheric ozone (DU)

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. For GFDL-emi 10H2 is H2 perturbation and 10CH4 is H2+CH4 pert minus the H2 pert.

In [24]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.04,0.03,0.03,0.70,0.06,0.03,0.12
10CH4,0.83,0.88,0.53,0.87,nan,0.73,0.66


### 3.2 Change in stratospheric ozone (DU)

In [25]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.01,-0.01,-0.04,-0.38,0.01,nan,-0.07
10CH4,0.83,0.80,0.25,1.41,nan,nan,0.78


### 3.5 Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone concentration. For GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4 pert minus the H2 pert.

In [26]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,1.86,1.75,1.06,31.0,3.06,1.20,6.40
10CH4,41.0,42.7,22.1,45.4,nan,25.6,33.4


In [27]:
df_ozone_rf['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_rf['OSLOCTM3'].loc['10CH4'] 
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,1.86,1.75,1.06,31.0,3.06,1.20,6.40
10CH4,41.0,42.7,22.1,45.4,41.0,25.6,33.4


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline. Preliminary results for OsloCTM3-emis.

In [28]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi,GFDL_nudge
Scenario,,,,,
10H2,1.48,0.51,0.82,2.43,4.92
10CH4,10.1,3.76,6.62,nan,20.0


In [29]:
df_h2o_rf['OSLOCTM3-emi'].loc['10CH4'] = df_h2o_rf['OSLOCTM3'].loc['10CH4'] 
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi,GFDL_nudge
Scenario,,,,,
10H2,1.48,0.51,0.82,2.43,4.92
10CH4,10.1,3.76,6.62,10.1,20.0


In [30]:
file = 'H2O_rf_gfdl.txt'
h2o_rf_gfdl = pd.read_csv(path + file, sep=',',index_col=0,header=0)
h2o_rf_gfdl

,GFDL
H2 [mW m-2],32.6
H2+CH4 [mW m-2],51.7


## 5. Aerosol RF [mW m-2]

In [31]:
file = 'aerosol_rf.txt'
df_aerosol_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_aerosol_rf

,OSLOCTM3,GFDL-emi,GFDL_nudge,OSLOCTM3-emi,INCA
Scenario,,,,,
10H2,-0.28,-21.8,0.47,-0.39,0.22
10CH4,-1.11,-14.5,0.18,nan,2.60


In [32]:
df_aerosol_rf['OSLOCTM3-emi'].loc['10CH4'] = df_aerosol_rf['OSLOCTM3'].loc['10CH4'] 
df_aerosol_rf

,OSLOCTM3,GFDL-emi,GFDL_nudge,OSLOCTM3-emi,INCA
Scenario,,,,,
10H2,-0.28,-21.8,0.47,-0.39,0.22
10CH4,-1.11,-14.5,0.18,-1.11,2.60



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [33]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3       6.23
WACCM6-2deg    5.36
INCA           7.36
GFDL-emi       6.96
OSLOCTM3-emi   6.48
UKCA           4.74
GFDL_nudge     6.98
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above, or move description here)... Why this includes feedback...

In [34]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

OSLOCTM3       0.31
WACCM6-2deg    0.33
INCA           0.29
GFDL-emi       0.21
OSLOCTM3-emi   0.32
UKCA           0.24
GFDL_nudge     0.26
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [35]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           3.67
GFDL-emi        nan
OSLOCTM3-emi    nan
UKCA           3.80
GFDL_nudge     4.18
Name: surf_ch4_per_ch4_flux, dtype: float64

Set equal to OsloCTM3 concentrationdriven

In [36]:
df_surf_ch4_per_ch4_flux['OSLOCTM3-emi'] = df_surf_ch4_per_ch4_flux['OSLOCTM3']
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           3.67
GFDL-emi        nan
OSLOCTM3-emi   3.46
UKCA           3.80
GFDL_nudge     4.18
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [37]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv (REF Paolot)
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux

OSLOCTM3       1.08
WACCM6-2deg    1.14
INCA           1.08
GFDL-emi       1.00
OSLOCTM3-emi   1.11
UKCA           0.93
GFDL_nudge     1.09
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

Why we multiply by -1

In [38]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi    nan
UKCA           0.02
GFDL_nudge     0.01
Name: h2_flux_per_ch4_flux, dtype: float64

In [39]:
df_h2_flux_per_ch4_flux['OSLOCTM3-emi'] = df_h2_flux_per_ch4_flux['OSLOCTM3']
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi   0.01
UKCA           0.02
GFDL_nudge     0.01
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

Explain why this is AGWP

In [40]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'





### Initialize H2 GWP table

In [41]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])

### Add methane GWP

In [42]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [43]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_h2o['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2+CH4 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'

#NBNB GFDL methane induced is included.

In [44]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'
df_h2o_rf_per_h2_flux['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux (Move this to the methane part?)
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [45]:
df_h2o_rf_per_ch4_flux['OSLOCTM3-emi'] = df_h2o_rf_per_ch4_flux['OSLOCTM3']
df_h2o_rf_per_ch4_flux

GFDL-emi        nan
GFDL_nudge     0.46
INCA           0.13
OSLOCTM3       0.19
OSLOCTM3-emi   0.19
UKCA            nan
WACCM6-2deg    0.07
Name: h2o_rf_per_ch4_flux, dtype: float64

### Add stratospheric H2O GWP

In [46]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2

### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [47]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = (df_ozone_rf['GFDL-emi'].loc['10H2']+df_ozone_rf['GFDL-emi'].loc['10CH4'])/df_h2_flux['GFDL-emi'].loc['deltaH2']


In [48]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

In [49]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

In [50]:
df_ozone_rf_per_ch4_flux['OSLOCTM3-emi'] = df_ozone_rf_per_ch4_flux['OSLOCTM3']
df_ozone_rf_per_ch4_flux

OSLOCTM3       0.78
WACCM6-2deg    0.81
INCA           0.45
GFDL-emi        nan
OSLOCTM3-emi   0.78
UKCA           0.54
GFDL_nudge     0.77
Name: ozone_rf_per_ch4_flux, dtype: float64

### Add Ozone GWP

In [51]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2

### For the per flux table

In [52]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'



In [53]:
df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

### HYDROGEN AGWP100 aerosol [mW m-2 yr Tg-1]

In [54]:
df_h2_agwp_aerosol = df_aerosol_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_aerosol.name = 'h2_agwp_aerosol'
df_h2_agwp_aerosol
#NBNB GFDL-emi include methane induced.

GFDL-emi       -0.11
GFDL_nudge      0.02
INCA            0.03
OSLOCTM3       -0.03
OSLOCTM3-emi   -0.03
UKCA             nan
WACCM6-2deg      nan
Name: h2_agwp_aerosol, dtype: float64

In [55]:
df_h2_gwp.loc['aerosol'] = df_h2_agwp_aerosol/agwp100_CO2 


In [56]:
#Add to the flux table
df_aerosol_rf_per_h2_flux = df_aerosol_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_aerosol_rf_per_h2_flux.name= 'aerosol_rf_per_h2_flux'


df_aerosol_rf_per_ch4_flux = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_aerosol_rf_per_ch4_flux.name = 'aerosol_rf_per_ch4_flux'

In [57]:
df_aerosol_rf_per_ch4_flux['OSLOCTM3-emi']=df_aerosol_rf_per_ch4_flux['OSLOCTM3']

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden, tropospheric concentration?

In [58]:
df_ch4_surfconc['OSLOCTM3-emi'].loc['deltaCH4']=df_ch4_surfconc['OSLOCTM3'].loc['deltaCH4']

In [59]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'


### Add methane induced O3 GWP

In [60]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.19,2.34
CH4,5.35,5.64,5.34,4.95,5.47,4.60,5.40
strat H2O,1.94,0.58,1.27,2.93,1.97,nan,1.80
O3 CH4ind,2.73,3.00,1.47,nan,2.80,1.47,2.24
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan
aerosol,-0.37,nan,0.34,-1.24,-0.32,nan,0.17


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [61]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'

### Add methane induced strat H2O GWP

In [62]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.19,2.34
CH4,5.35,5.64,5.34,4.95,5.47,4.60,5.40
strat H2O,1.94,0.58,1.27,2.93,1.97,nan,1.80
O3 CH4ind,2.73,3.00,1.47,nan,2.80,1.47,2.24
strat H2O CH4ind,0.67,0.27,0.44,nan,0.69,nan,1.34
aerosol,-0.37,nan,0.34,-1.24,-0.32,nan,0.17


### HYDROGEN AGWP100 methane induced aerosols [mW m-2 yr Tg-1]

In [63]:
df_h2_agwp_ch4ind_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_aerosol.name = 'h2_agwp_ch4ind_aerosols'
df_h2_agwp_ch4ind_aerosol

GFDL-emi         nan
GFDL_nudge      0.00
INCA            0.02
OSLOCTM3       -0.01
OSLOCTM3-emi   -0.01
UKCA             nan
WACCM6-2deg      nan
Name: h2_agwp_ch4ind_aerosols, dtype: float64

### Hydrogen GWP including aerosols

In [64]:
df_h2_gwp.loc['aerosol CH4ind'] = df_h2_agwp_ch4ind_aerosol/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.19,2.34
CH4,5.35,5.64,5.34,4.95,5.47,4.60,5.40
strat H2O,1.94,0.58,1.27,2.93,1.97,nan,1.80
O3 CH4ind,2.73,3.00,1.47,nan,2.80,1.47,2.24
strat H2O CH4ind,0.67,0.27,0.44,nan,0.69,nan,1.34
aerosol,-0.37,nan,0.34,-1.24,-0.32,nan,0.17
aerosol CH4ind,-0.07,nan,0.17,nan,-0.08,nan,0.01


In [65]:
#Not include the aerosol GWP in the main table. Drop them here
df_h2_gwp = df_h2_gwp.drop(['aerosol','aerosol CH4ind'])
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.19,2.34
CH4,5.35,5.64,5.34,4.95,5.47,4.60,5.40
strat H2O,1.94,0.58,1.27,2.93,1.97,nan,1.80
O3 CH4ind,2.73,3.00,1.47,nan,2.80,1.47,2.24
strat H2O CH4ind,0.67,0.27,0.44,nan,0.69,nan,1.34


# Methane GWP

Initialize CH4 GWP

In [66]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
#df_ch4_gwp

### Methane AGWP100 O3 [mW m-2 yr Tg-1]

Integrated O3 from CH4 RF [W m-2 yr Tg-1]

In [67]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'



### Add ozone GWP

In [68]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [69]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'

### Add methane GWP

In [70]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [71]:
print(df_h2o_rf.loc['10CH4'])
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

OSLOCTM3       10.1
WACCM6-2deg    3.76
INCA           6.62
OSLOCTM3-emi   10.1
GFDL_nudge     20.0
Name: 10CH4, dtype: float64


### Add Strat H2O GWP:

In [72]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Methane AGWP100 aerosols [mW m-2 yr Tg-1]

In [73]:
df_ch4_agwp_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_agwp_aerosol.name = 'ch4_agwp_aerosol'
df_ch4_agwp_aerosol
#NBNB GFDL methane induced is included.

GFDL-emi         nan
GFDL_nudge      0.00
INCA            0.05
OSLOCTM3       -0.02
OSLOCTM3-emi     nan
UKCA             nan
WACCM6-2deg      nan
Name: ch4_agwp_aerosol, dtype: float64

### Add Aerosol GWP

In [74]:
df_ch4_gwp.loc['aerosol'] = df_ch4_agwp_aerosol/agwp100_CO2
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,8.76,9.10,4.99,nan,8.76,6.02,8.59
CH4,17.1,17.1,18.2,nan,17.1,18.8,20.7
strat H2O,2.16,0.80,1.50,nan,2.16,nan,5.15
H2,nan,nan,nan,nan,nan,nan,nan
aerosol,-0.24,nan,0.59,nan,nan,nan,0.05


In [75]:
#And drop the aerosol GWP in the main table:
df_ch4_gwp = df_ch4_gwp.drop(['aerosol'])
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,8.76,9.10,4.99,nan,8.76,6.02,8.59
CH4,17.1,17.1,18.2,nan,17.1,18.8,20.7
strat H2O,2.16,0.80,1.50,nan,2.16,nan,5.15
H2,nan,nan,nan,nan,nan,nan,nan


### Add GWP via H2

In [76]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()

# Part III: Main results and tables

## H2 GWP 100

In [77]:
model_dict = { 'OSLOCTM3':'OsloCTM',
               'WACCM6-2deg':'WACCM',
               'INCA':'INCA',
               'GFDL-emi':'GFDL-emi',
               'GFDL_nudge':'GFDL',
               'UKCA':'UKCA',
               'OSLOCTM3-emi':'OsloCTM-emi'}

sorted_array = ['GFDL','OsloCTM','INCA','UKCA','WACCM','GFDL-emi','OsloCTM-emi']
sorted_array_2 = ['GFDL','OsloCTM','INCA','UKCA','WACCM','Model mean','GFDL-emi','OsloCTM-emi']

In [78]:
#Save to file below. Where CH4 are split based on feedback factor:
#print(outputpath + 'table_h2_gwp.csv')
#df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')
#df_h2_gwp = df_h2_gwp[sorted(df_h2_gwp.columns)]

df_h2_gwp = df_h2_gwp[sorted(df_h2_gwp.columns)]

df_h2_gwp.loc['total']=df_h2_gwp.sum()

df_h2_gwp_table = df_h2_gwp.copy()

df_h2_gwp_table.rename(model_dict,axis=1,inplace=True)
df_h2_gwp_table = df_h2_gwp_table[sorted_array]
df_h2_gwp_table.T


/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,O3,CH4,strat H2O,O3 CH4ind,strat H2O CH4ind,total
GFDL,2.34,5.40,1.80,2.24,1.34,13.1
OsloCTM,2.44,5.35,1.94,2.73,0.67,13.1
INCA,1.64,5.34,1.27,1.47,0.44,10.2
UKCA,1.19,4.60,nan,1.47,nan,7.26
WACCM,1.97,5.64,0.58,3.00,0.27,11.5
GFDL-emi,4.34,4.95,2.93,nan,nan,12.2
OsloCTM-emi,2.49,5.47,1.97,2.80,0.69,13.4


## CH4 GWP 100

In [79]:
df_ch4_gwp = df_ch4_gwp[sorted(df_ch4_gwp.columns)]
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp.T

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,O3,CH4,strat H2O,H2,total
GFDL-emi,nan,nan,nan,nan,0.00
GFDL_nudge,8.59,20.7,5.15,0.16,34.6
INCA,4.99,18.2,1.50,0.10,24.8
OSLOCTM3,8.76,17.1,2.16,0.19,28.2
OSLOCTM3-emi,8.76,17.1,2.16,0.19,28.2
UKCA,6.02,18.8,nan,0.11,24.9
WACCM6-2deg,9.10,17.1,0.80,0.19,27.2


## Table per flux H2

In [80]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],
                            df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,
                            df_ch4_flux_per_h2_flux,
                            df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux*1000.,
                            df_strat_du_ozone_per_h2_flux*1000.,
                            df_ozone_rf_per_h2_flux,
                            df_h2o_rf_per_h2_flux,
                            df_aerosol_rf_per_h2_flux],axis=1, sort=False)

#Save to file:
df_per_flux_h2 = df_per_flux_h2.sort_index()
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2.rename(model_dict,axis=0,inplace=True)
df_per_flux_h2.loc['Model mean'] = df_per_flux_h2.drop(['GFDL-emi','OsloCTM-emi']).mean()
df_per_flux_h2['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
df_per_flux_h2=df_per_flux_h2.reindex(sorted_array_2)
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [10$^{-3}$ DU yr/Tg],Strat. ozone per flux [10$^{-3}$ DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
GFDL,30.5,6.98,1.09,0.26,0.48,4.09,-2.42,0.21,0.16,0.02
OsloCTM,8.54,6.23,1.08,0.31,0.48,4.48,0.77,0.22,0.17,-0.03
INCA,7.23,7.36,1.08,0.29,0.48,3.93,-5.56,0.15,0.11,0.03
UKCA,11.3,4.74,0.93,0.24,0.41,2.94,nan,0.11,nan,nan
WACCM,9.93,5.36,1.14,0.33,0.50,3.31,-1.10,0.18,0.05,nan
Model mean,nan,6.13,1.06,0.29,0.47,3.75,-2.08,0.17,0.13,0.00
GFDL-emi,197,6.96,1.00,0.21,0.44,3.54,-1.93,0.16,0.17,-0.11
OsloCTM-emi,13.8,6.48,1.11,0.32,0.49,4.61,0.75,0.22,0.18,-0.03


## Table per flux CH4

In [81]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],
                            df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,
                            df_trop_du_ozone_per_ch4_flux*1000.,
                            df_strat_du_ozone_per_ch4_flux*1000.,
                            df_ozone_rf_per_ch4_flux,
                            df_h2o_rf_per_ch4_flux,
                            df_aerosol_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
df_per_flux_ch4 = df_per_flux_ch4.sort_index()
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_ch4_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:

df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])

df_per_flux_ch4.rename(model_dict, inplace=True)

df_per_flux_ch4.loc['Model mean'] = df_per_flux_ch4.drop(['GFDL-emi','OsloCTM-emi']).mean()

df_per_flux_ch4 = df_per_flux_ch4.reindex(sorted_array_2)

df_per_flux_ch4          

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],Trop. ozone per flux [10$^{-3}$ DU yr/Tg],Strat. ozone per flux [10$^{-3}$ DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
GFDL,43.4,4.18,0.01,15.3,17.9,0.77,0.46,0.00
OsloCTM,52.4,3.46,0.01,15.9,15.8,0.78,0.19,-0.02
INCA,49.4,3.67,0.01,10.8,5.07,0.45,0.13,0.05
UKCA,47.6,3.80,0.02,15.4,nan,0.54,nan,nan
WACCM,52.4,3.45,0.02,16.7,15.3,0.81,0.07,nan
Model mean,49.0,3.71,0.01,14.8,13.5,0.67,0.21,0.01
GFDL-emi,nan,nan,nan,nan,nan,nan,nan,nan
OsloCTM-emi,nan,3.46,0.01,15.9,15.8,0.78,0.19,-0.02


## H2 budget table

In [84]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],
                          df_h2_drydep.loc['CTRL'],
                          df_h2_estemis, 
                          df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_soil_sink_lifetime.loc['CTRL'],
                          df_h2_lifetime.loc['CTRL']
                         ],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2 = df_budget_h2.sort_index()
df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2



,H2 burden [Tg],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
GFDL-emi,192,22.3,45.1,54.6,31.9,8.62,3.52,2.50
GFDL_nudge,188,21.7,45.0,56.6,33.3,8.66,3.32,2.40
INCA,195,22.5,47.1,52.0,27.3,8.66,3.74,2.61
OSLOCTM3,196,28.4,56.3,59.5,31.6,6.91,3.30,2.23
OSLOCTM3-emi,209,30.1,56.2,58.4,32.2,6.95,3.58,2.36
UKCA,191,26.8,49.4,88.7,66.0,7.15,2.16,1.66
WACCM6-2deg,195,29.0,33.9,73.0,68.1,6.70,2.67,1.91


## CH4 budget table

In [ ]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],
                           df_ch4_loss.loc['CTRL'],
                           df_ch4_lifetime.loc['CTRL']
                           ],
                          axis=1)
df_budget_ch4 = df_budget_ch4.sort_index()


df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 chem loss OH [Tg/yr]','CH4 lifetime [yrs]'] 

df_budget_ch4.to_csv(outputpath + 'table_budget_ch4.csv')

df_budget_ch4

# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [ ]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

Add explanation... 

In [ ]:
df_w_o_feedback =df_ch4_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [ ]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4 = df_feedback_factor_ch4.sort_index()
df_feedback_factor_ch4.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4

Split the CH4 GWP into direkt and indirect based on the feedback factor. For the GWP figure

In [ ]:
feedback_factor = df_feedback_factor_ch4
feedback_frac = 1.0 - (1.0/feedback_factor)
feedback_frac.name = 'feedback_frac'
feedback_frac

In [ ]:
#Save to file:

df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-feedback_frac)
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*feedback_frac

df_h2_gwp['GFDL-emi'].loc['CH4dir'] = df_h2_gwp['GFDL-emi'].loc['CH4']
df_h2_gwp['OSLOCTM3-emi'].loc['CH4dir'] = df_h2_gwp['OSLOCTM3-emi'].loc['CH4']
df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [ ]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [ ]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

In [ ]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

### Change in lifetime per flux

In [ ]:
df_ch4_lifetime.loc['deltaH2'] = df_ch4_lifetime.loc['10H2']-df_ch4_lifetime.loc['CTRL']
df_ch4_lifetime.loc['deltaCH4'] = df_ch4_lifetime.loc['10CH4']-df_ch4_lifetime.loc['CTRL']
df_ch4_lifetime

In [ ]:
#Direct (changes in methane lifetime per h2 flux [days per Tg H2])
df_ch4_lifetime_per_h2_flux =  df_ch4_lifetime.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_lifetime_per_h2_flux*365.0 #Days

In [ ]:
#Indirect (changes in methane lifetime per h2 flux [days per Tg H2] due to changes in methane):
df_ch4_lifetime_per_ch4_flux =  df_ch4_lifetime.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_lifetime_per_ch4_flux*365.0*df_ch4_flux_per_h2_flux
